In [6]:
import pqr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf


# Выгрузка исходных данных для построения стратегии или модели


In [7]:
import requests

voluminous_stocks_tickers = requests.get(
    "https://github.com/aalitvinov/ITfF_tomtosov/raw/main/hw_data/voluminous_stocks_tickers.json"
).json()


In [59]:
from datetime import datetime

start = datetime(2008, 1, 1)
end = datetime(2023, 1, 1)


In [87]:
snp500 = yf.download(
            tickers="^GSPC",
            interval="1d",
            start=start,
            end=end,
            show_errors=False,
        )

[*********************100%***********************]  1 of 1 completed


In [88]:
try:
    voluminous_stocks_yf = pd.read_csv(
        "./hw_data/voluminous_stocks_yf.csv.xz",
        compression="xz",
        header=[0, 1],
        index_col=[0, 1],
    )
    snp500 = pd.read_csv("./hw_data/snp500_historical.csv")
except FileNotFoundError:
    voluminous_stocks_yf: pd.DataFrame = (
        yf.download(
            tickers=voluminous_stocks_tickers,
            interval="1d",
            start=start,
            end=end,
            group_by="column",
            progress=True,
            show_errors=False,
        )
        .loc[:, ["Adj Close", "Volume"]]
        .rename({"Adj Close": "adj_close", "Volume": "volume"}, axis=1)
    )
    snp500: pd.DataFrame = (
        yf.download(
            tickers="^GSPC",
            interval="1d",
            start=start,
            end=end,
            show_errors=False,
        )
        .loc[:, ["Adj Close", "Volume"]]
        .rename({"Adj Close": "adj_close", "Volume": "volume"}, axis=1)
    )
    snp500.to_csv("./hw_data/snp500_historical.csv")
    voluminous_stocks_yf.to_csv(
        "./hw_data/voluminous_stocks_yf.csv.xz", compression="xz", index=True
    )


[*********************100%***********************]  200 of 200 completed
[*********************100%***********************]  1 of 1 completed


In [89]:
voluminous_stocks_yf.head(2)

adj_close                                             \
                            AA        AAL       AAPL       ABBV      ABEV   
Date                                                                        
2019-04-18 00:00:00  26.358833  33.915295  49.325840  64.328300  3.760329   
2019-04-22 00:00:00  27.822117  33.037067  49.487942  64.809303  3.820701   

                                                                           \
                           ABT        AFL        AIG       AIV       AMAT   
Date                                                                        
2019-04-18 00:00:00  69.277443  44.319912  41.089413  4.561910  42.011478   
2019-04-22 00:00:00  69.830368  43.993774  41.053711  4.492435  41.858532   

                     ...   volume                                           \
                     ...     VXRT        VZ WLL      WMB      WMT      WPM   
Date                 ...                                                     
2019-04-18 00:00:00  ...   907700  13189000 NaN  6097400  3727800  3175300   
2019-04-22 00:00:00  ...  1207900  11950400 NaN  4138800  5079400  1466600   

                                                          
                            X       XOM XSPA          ZM  
Date                                                      
2019-04-18 00:00:00  11889100   9493900  NaN  25764700.0  
2019-04-22 00:00:00  10015800  10770600  NaN   9949700.0  

[2 rows x 400 columns]

In [62]:
voluminous_stocks_yf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3778 entries, 2019-04-18 00:00:00 to 2019-04-17 00:00:00
Columns: 400 entries, ('adj_close', 'AA') to ('volume', 'ZM')
dtypes: float64(227), int64(173)
memory usage: 11.6+ MB


In [11]:
len(voluminous_stocks_tickers)


200

# Обработка финансовых данных


Удаление столбцов с акциями, у которых нет хотя бы 500 значений:

In [77]:
voluminous_stocks_yf.dropna(axis=1, thresh=500).shape

(3778, 382)

In [65]:
prices = voluminous_stocks_yf.loc[:, "adj_close"]
prices.head(3)

,AA,AAL,AAPL,ABBV,ABEV,ABT,AFL,AIG,AIV,AMAT,...,VXRT,VZ,WLL,WMB,WMT,WPM,X,XOM,XSPA,ZM
Date,,,,,,,,,,,,,,,,,,,,,
2019-04-18 00:00:00,26.358835,33.915295,49.325844,64.328308,3.760329,69.277428,44.319916,41.089409,4.561911,42.011478,...,0.77,47.840244,NaN,21.909897,96.521996,20.805519,15.938238,65.185242,NaN,62.000000
2019-04-22 00:00:00,27.822113,33.037071,49.487949,64.809296,3.820701,69.830391,43.993767,41.053707,4.492435,41.858528,...,0.76,48.112247,NaN,22.250301,95.764267,20.644463,15.637338,66.607391,NaN,65.699997
2019-04-23 00:00:00,28.207712,33.520584,50.201721,65.232231,3.881073,71.320526,44.310860,41.035854,4.577871,41.896770,...,0.75,47.106651,NaN,22.296721,96.419090,20.483391,15.676164,66.993042,NaN,69.000000


In [81]:
volume = voluminous_stocks_yf.loc[:, "volume"] * prices
volume.head(3)

,AA,AAL,AAPL,ABBV,ABEV,ABT,AFL,AIG,AIV,AMAT,...,VXRT,VZ,WLL,WMB,WMT,WPM,X,XOM,XSPA,ZM
Date,,,,,,,,,,,,,,,,,,,,,
2019-04-18 00:00:00,2.497816e+08,1.125377e+08,4.773913e+09,5.007894e+08,9.319599e+07,6.728362e+08,1.893302e+08,1.532347e+08,4.784527e+07,2.571985e+08,...,698928.982687,6.309650e+08,NaN,1.335934e+08,3.598147e+08,6.606376e+07,1.894913e+08,6.188622e+08,NaN,1.597411e+09
2019-04-22 00:00:00,1.736434e+08,1.283226e+08,3.848084e+09,3.472871e+08,9.241320e+07,4.057635e+08,9.828207e+07,1.080123e+08,4.799276e+07,1.955714e+08,...,918003.988481,5.749606e+08,NaN,9.208955e+07,4.864250e+08,3.027717e+07,1.566204e+08,7.174016e+08,NaN,6.536953e+08
2019-04-23 00:00:00,1.349683e+08,1.116168e+08,4.683419e+09,6.471429e+08,7.855603e+07,4.866556e+08,1.548665e+08,2.066730e+08,3.378605e+07,2.808718e+08,...,441975.000000,1.106474e+09,NaN,1.211069e+08,4.869935e+08,3.884880e+07,1.811976e+08,7.097980e+08,NaN,4.682685e+08


In [82]:
prices, volume = pqr.utils.align(prices, volume) # type: ignore
prices, volume = pqr.utils.replace_with_nan(prices, volume, to_replace=0) # type: ignore
prices: pd.DataFrame = prices
volume: pd.DataFrame = volume

In [83]:
universe = (prices > 10) & (volume > 20_000_000)

In [86]:
universe.sum().sort_values(ascending=False) # type: ignore

KO      3778
BAX     3778
GS      3778
KSS     3778
GILD    3778
        ... 
GOED       0
NILE       0
CDEV       0
NLOK       0
PIRS       0
Length: 200, dtype: int64

# Построение инвестиционной стратегии или финансовой модели


In [90]:
returns_calculator = pqr.utils.partial(
    pqr.calculate_returns, 
    universe_returns=pqr.prices_to_returns(prices),
)

In [96]:
ew_benchmark = pqr.Benchmark.from_universe(
    universe=universe,
    allocator=pqr.equal_weights,
    calculator=returns_calculator,
    name='EW_Benchmark'
)

ew_benchmark.returns

EW_Benchmark
2019-04-18    0.000000
2019-04-22   -0.000187
2019-04-23    0.008288
2019-04-24   -0.001217
2019-04-25   -0.003453
                ...   
2019-04-11   -0.001882
2019-04-12    0.006134
2019-04-15   -0.003396
2019-04-16    0.004152
2019-04-17   -0.001348
Length: 3778, dtype: float64

In [95]:
imoex_benchmark = pqr.Benchmark.from_index(
    index=snp500.loc[:, "adj_close"],
    name='S&P 500'
)

imoex_benchmark.returns

S&P 500
2007-12-31    0.000000
2008-01-02   -0.014438
2008-01-03    0.000000
2008-01-04   -0.024552
2008-01-07    0.003223
                ...   
2022-12-23    0.005868
2022-12-27   -0.004050
2022-12-28   -0.012021
2022-12-29    0.017461
2022-12-30   -0.002541
Name: adj_close, Length: 3778, dtype: float64

# Аналитика и визуализация результатов
